<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_experimental_EDA_lines_heft_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [2]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [3]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Downloads

In [3]:
!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/L30b3SynSzMgdQ
!unrar e ./data.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from ./data.rar

Extracting  data.csv                                                       1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55%  OK 
Extracting  data.RDS                                                      56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92%

In [4]:
!wc -l ./data.csv

14248527 ./data.csv


### Imports

In [4]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.3.5
1.21.6


In [5]:
np.iinfo(np.int8).max, np.iinfo(np.int16).max, np.iinfo(np.int32).max

(127, 32767, 2147483647)

In [6]:
np.finfo(np.float16).precision, np.finfo(np.float32).precision, np.finfo(np.float64).precision

(3, 6, 15)

In [7]:
import os, psutil
from tqdm import tqdm
import gc

### Code

#####Functions

In [8]:
def run_bash(bashCommand:str, nameCommand = ''):
        process = subprocess.Popen([bashCommand], 
                           shell=True)
        _, error = process.communicate()
        if error:
            print(f'{nameCommand} error:\n', error)

In [9]:
# Присоединяем итоговый результат и по первому тайму
def add_match_results (data_df:pd.DataFrame, cols = ['Id', 'Result1', 'Result2', 'Periods'], info_path = './info.csv'):
    info_df = pd.read_csv(info_path, sep = ';', usecols = cols)
    info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)
    info_df[['Time1Res1', 'Time1Res2']] = info_df['Period1'].str.split(':', expand = True)
    info_df = info_df[~info_df['Id'].duplicated(keep = False)]
    period1_result_dict = info_df.set_index('Id')[['Time1Res1', 'Time1Res2', 'Result1', 'Result2']].to_dict(orient = 'index')
    #data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] =  \
    return [[
        #period1_result_dict[id]['Time1Res1'], period1_result_dict[id]['Time1Res1'], 
        period1_result_dict[id]['Result1'], period1_result_dict[id]['Result2']] 
    if id in period1_result_dict else [None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]
    

In [10]:
# Добавляем прематчевые линии
def add_match_lines (data_df:pd.DataFrame, cols = ['P1', 'P2'], prem_path = './prem.csv'):
    prem_df = pd.read_csv(prem_path, sep = ';')
    prem_df = prem_df[~prem_df['Id'].duplicated(keep = False)]
    prem_dict = prem_df.set_index('Id')[cols].to_dict(orient = 'index')
    #data_df.loc[:,['P1', 'PX', 'P2', 'PR']] =  \
    return [[prem_dict[id]['P1'], prem_dict[id]['P2']] 
    if id in prem_dict else [None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]

In [11]:
def transform_dataset(data_df:pd.DataFrame, remain = []):
    new_match_vector = data_df['Id'] != data_df['Id'].shift(1)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df[data_df['Score1'].isna() & new_match_vector] = 0
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df[data_df['Score2'].isna() & new_match_vector] = 0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    new_match_vector5 = data_df['Id'] != data_df['Id'].shift(5)
    data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A1relativ']] = np.float32(0.0)
    data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
    data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
    data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A2relativ']] =  np.float32(0.0)
    data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
    data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту    
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA1relativ']] = np.float32(0.0)
    data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
    data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
    data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA2relativ']] = np.float32(0.0)
    data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
    data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df[data_df['Pos1'].isna() & new_match_vector] = 0
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df[data_df['Pos2'].isna() & new_match_vector] = 0
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df[data_df['Off1'].isna() & new_match_vector] = 0
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df[data_df['Off2'].isna() & new_match_vector] = 0
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df[data_df['On1'].isna() & new_match_vector] = 0
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df[data_df['On2'].isna() & new_match_vector] = 0
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)    
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df[data_df['YC1'].isna() & new_match_vector] = 0
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df[data_df['YC2'].isna() & new_match_vector] = 0
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df[data_df['RC1'].isna() & new_match_vector] = 0
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df[data_df['RC2'].isna() & new_match_vector] = 0
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df[data_df['Sub1'].isna() & new_match_vector] = 0
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df[data_df['Sub2'].isna() & new_match_vector] = 0
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df[data_df['Cor1'].isna() & new_match_vector] = 0
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df[data_df['Cor2'].isna() & new_match_vector] = 0
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

#### Devide fo 6 parts (train, test, holdout -> 2 parts)

In [12]:
data_types_dict = {
    'Id':np.int32, 
    'StatTime':np.datetime64, 
    'Minute':np.int8, 
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8, 
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8, 
    'Time':np.datetime64, 
    'Minute.1':np.int8, 
    'RawTime':np.datetime64, 
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8, 
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16, 
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16 
}

In [13]:
drop_cols = [
    #'StatTime', 
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period'
]

In [14]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]

In [20]:
data_df = pd.DataFrame()

In [19]:
for col, col_type in data_types_dict.items(): #{'Id':np.int32 , 'StatTime':np.datetime64}.items(): #
    if col not in drop_cols:
        data_col = pd.read_csv('./data.csv', sep = ';', usecols = [col])
        if col == 'Id':
            if data_col[col].isna().sum() > 0:
                print(col , ' nan error')
            data_col = data_col.astype(col_type)
            new_match_vector = data_col['Id'] != data_col['Id'].shift(1)
            data_df[col] = data_col
        elif col == 'StatTime':
            data_col.loc[data_col['StatTime'] == 'NA                 ',['StatTime']] = '31.01.2010 00:00:00'
            data_col = data_col.astype(col_type)
            data_df[col] = data_col
        elif col in k_cols:
            data_col = data_col.fillna(0).astype(col_type)
            data_df[col] = data_col             
        else:
            data_col[data_col[col].isna() & new_match_vector] = 0
            data_col = data_col.fillna(method = 'ffill').astype(col_type)
            data_df[col] = data_col
        del data_col
        gc.collect()
        #---------------
        process = psutil.Process(os.getpid())
        print(col, ' mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
        #---------------    

Id  mem usage:  0.43 GiB


KeyboardInterrupt: ignored

In [16]:
time_df = data_df.drop_duplicates()
time_df.to_csv('./time.csv', index = False)

In [18]:
## Добавляем финальный результат
data_df.loc[:, ['Result1', 'Result2']] = add_match_results (data_df)
data_df[['Result1', 'Result2']] = data_df[['Result1', 'Result2']].astype(np.int8)

## Добавляем линии
data_df.loc[:,['P1', 'P2']] = add_match_lines (data_df)
data_df[['P1', 'P2']] = data_df[['P1', 'P2']].astype(np.float16)
#---------------
process = psutil.Process(os.getpid())
print(col, ' mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

100%|██████████| 14248526/14248526 [01:36<00:00, 147028.59it/s]


H2  mem usage:  1.44 GiB


In [19]:
data_df = transform_dataset(data_df, #) 
    remain = ['Score1', 'Score2'])

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


12. Линии посчитаны...


In [20]:
print('P1 NaN & P2 NaN: ', data_df['P1_transformed'].isna().sum(), data_df['P2_transformed'].isna().sum())

data_df = data_df[~data_df['P1_transformed'].isna()]
data_df = data_df[~data_df['P2_transformed'].isna()]

print('Score-Result error 1&2: ', (~(data_df['Result1'] - data_df['Score1']) >= 0).sum(), (~(data_df['Result1'] - data_df['Score1']) >= 0).sum())
data_df = data_df.loc[(data_df['Result1'] - data_df['Score1']) >= 0]
data_df = data_df.loc[(data_df['Result2'] - data_df['Score2']) >= 0]


#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

P1 NaN & P2 NaN:  105537 105537
Score-Result error 1&2:  840 840
mem usage:  4.89 GiB


In [21]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14141621 entries, 0 to 14248525
Data columns (total 59 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Id                int32  
 1   Score1            int8   
 2   Score2            int8   
 3   Active.1          int8   
 4   W1                float16
 5   WX                float16
 6   W2                float16
 7   X1                float16
 8   X2                float16
 9   W12               float16
 10  TotalValue        float16
 11  Over              float16
 12  Under             float16
 13  Hand1Value        float16
 14  H1                float16
 15  H2                float16
 16  Result1           int8   
 17  Result2           int8   
 18  min_norm          float32
 19  Score1_norm       float32
 20  Score2_norm       float32
 21  Score_diff        float32
 22  Score_cat_1       uint8  
 23  Score_cat_2       uint8  
 24  Score_cat_3       uint8  
 25  Score_cat_4       uint8  
 26  Score_cat_5 

In [22]:
#data_df.to_csv('./data_all.csv', index = False)

In [23]:
#data_df = pd.read_csv('./data_all.csv',)

In [24]:
train_id_df = pd.read_csv('./train_id.csv', header = None, names = ['id'])
test_id_df = pd.read_csv('./test_id.csv', header = None, names = ['id'])
holdout_id_df = pd.read_csv('./holdout_id.csv', header = None, names = ['id'])

train_vector = data_df['Id'].isin(train_id_df['id'])
test_vector = data_df['Id'].isin(test_id_df['id'])
holdout_vector = data_df['Id'].isin(holdout_id_df['id'])

In [25]:
data_df.columns

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2', 'min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'A1relativ', 'A2relativ', 'DA1_scaled', 'DA2_scaled',
       'DA1perMIN', 'DA2perMIN', 'DA1relativ', 'DA2relativ', 'Pos1_cleaned',
       'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm', 'On2_norm',
       'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed'],
      dtype='object')

In [ ]:
cols = ['Id', 'BeginTime']
info_path = './info.csv'
info_df = pd.read_csv(info_path, sep = ';', 
                      usecols = cols
                      )


In [26]:
#%%writefile save_discription.txt
np.savez_compressed('./dataset', 
                    #id_train = id_train,
                    #id_test = id_test,
                    X_train =  data_df[data_df.columns[18:]][train_vector].values,
                    X_test = data_df[data_df.columns[18:]][test_vector].values,
                    X_holdout = data_df[data_df.columns[18:]][holdout_vector].values,
                    y_train_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[train_vector],
                    y_test_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[test_vector],
                    y_holdout_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[holdout_vector],
                    y_train_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[train_vector] + 1,
                    y_test_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[test_vector] + 1,
                    y_holdout_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[holdout_vector] + 1,
                    y_train_diff = (data_df['Result1'] - data_df['Result2']).values[train_vector],
                    y_test_diff = (data_df['Result1'] - data_df['Result2']).values[test_vector],
                    y_holdout_diff = (data_df['Result1'] - data_df['Result2']).values[holdout_vector],
                    y_train_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[train_vector],
                    y_train_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[train_vector],
                    y_test_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[test_vector],
                    y_test_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[test_vector],
                    y_holdout_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[holdout_vector],
                    y_holdout_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[holdout_vector],
                    score1_train = data_df['Score1'].values[train_vector],
                    score2_train = data_df['Score2'].values[train_vector],
                    result1_train = data_df['Result1'].values[train_vector],
                    result2_train = data_df['Result2'].values[train_vector],
                    score1_test = data_df['Score1'].values[test_vector],
                    score2_test = data_df['Score2'].values[test_vector],
                    result1_test = data_df['Result1'].values[test_vector],
                    result2_test = data_df['Result2'].values[test_vector],
                    score1_holdout = data_df['Score1'].values[holdout_vector],
                    score2_holdout = data_df['Score2'].values[holdout_vector],
                    result1_holdout = data_df['Result1'].values[holdout_vector],
                    result2_holdout = data_df['Result2'].values[holdout_vector],
                    K_train = data_df[data_df.columns[3:16]].values[train_vector],
                    K_test = data_df[data_df.columns[3:16]].values[test_vector],
                    K_holdout = data_df[data_df.columns[3:16]].values[holdout_vector]
                )

In [27]:
np.savez_compressed('./additional_data', 
                    id_train = data_df['Id'][train_vector].values,
                    id_test = data_df['Id'][test_vector].values,
                    id_holdout = data_df['Id'][holdout_vector].values,
                    min_train = data_df['min_norm'][train_vector].values,
                    min_test = data_df['min_norm'][test_vector].values,
                    min_holdout = data_df['min_norm'][holdout_vector].values
)

In [28]:
params = {}
params['description'] = 'датасет под бусты \n' + \
'удалены строки с nan в P1, P2 \n' + \
'простые фичи, \n' + \
'4 класса задач, бинарная классификация и мультикласс, и под регрессию 2 варианта '
params['features'] = list(data_df.columns)

In [23]:
data_version = 'football_live_npz_230131/'
project = neptune.init_project(
    project="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset'].upload('./dataset.npz')
project[data_version + 'description'].upload('./save_discription.txt')
project[data_version + 'additional_data'].upload('./additional_data.npz')
project[data_version + 'info'].upload('./info.csv')
project[data_version + 'time'].upload('./time.csv')
project[data_version + 'params'] = params
project.stop()

https://app.neptune.ai/scomesse/football/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api/project#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


In [ ]:
data_df.columns[:18]

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2'],
      dtype='object')

In [ ]:
data_df.columns

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2', 'min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'A1relativ', 'A2relativ', 'DA1_scaled', 'DA2_scaled',
       'DA1perMIN', 'DA2perMIN', 'DA1relativ', 'DA2relativ', 'Pos1_cleaned',
       'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm', 'On2_norm',
       'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed'],
      dtype='object')

In [ ]:
X_train = data_df[data_df.columns[18:]][train_vector].values
X_test = data_df[data_df.columns[18:]][test_vector].values
X_holdout = data_df[data_df.columns[18:]][holdout_vector].values

In [ ]:
y_train_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[train_vector]
y_test_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[test_vector]
y_holdout_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[holdout_vector])

In [ ]:
y_train_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[train_vector] + 1
y_test_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[test_vector] + 1
y_holdout_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[holdout_vector] + 1

In [ ]:
y_train_diff = (data_df['Result1'] - data_df['Result2']).values[train_vector]
y_test_diff = (data_df['Result1'] - data_df['Result2']).values[test_vector]
y_holdout_diff = (data_df['Result1'] - data_df['Result2']).values[holdout_vector]

In [ ]:
y_train_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[train_vector]
y_train_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[train_vector]
y_test_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[test_vector]
y_test_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[test_vector]
y_test_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[holdout_vector]
y_test_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[holdout_vector]

In [ ]:
X_train.shape, X_test.shape, X_holdout.shape

((11197708, 41), (2798988, 41), (144925, 41))

In [ ]:
data_df[data_df['Id'].isin(train_id_df['id'])].to_csv('data_train.csv', index = False)
data_df[data_df['Id'].isin(test_id_df['id'])].to_csv('data_test.csv', index = False)
data_df[data_df['Id'].isin(holdout_id_df['id'])].to_csv('data_holdout.csv', index = False)

In [ ]:
print()
data_df = data_df[data_df[['W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'Over', 'Under', 'H1', 'H2' ]].sum(axis = 1) > 0]
data_df = data_df[~(data_df['Active.1'] == 0)]

In [ ]:
len(data_df[data_df['Id'].isin(train_id_df['id'])]), \
len(data_df[data_df['Id'].isin(test_id_df['id'])]), \
len(data_df[data_df['Id'].isin(holdout_id_df['id'])])

(3375750, 849640, 124367)

In [ ]:
data_df[data_df['P1_transformed'].isna()]

In [ ]:
names = [
    'Id', 'StatTime', 'Minute', 'Active', 'Score1', 'Score2', 'A1', 'A2', 'DA1', 'DA2', 'Pos1', 'Pos2',
    'Off1', 'Off2', 'On1', 'On2', 'YC1', 'YC2', 'RC1', 'RC2', 'Sub1', 'Sub2', 'Pen1', 'Pen2',
    'Cor1', 'Cor2', 'Period', 'Comment', 'D', 'I', 'Active.1', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]


In [ ]:
data_train = data_train[data_train[['W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'Over', 'Under', 'H1', 'H2' ]].sum(axis = 1) > 0]

In [ ]:
data_train[~(data_train['Active.1'] == 0)]

In [ ]:
data_train.to_csv('./data_train.csv.gz', index = False, compression = {'method':'gzip'})
data_test.to_csv('./data_test.csv.gz', index = False, compression = {'method':'gzip'})
data_holdout.to_csv('./data_holdout.csv.gz', index = False, compression = {'method':'gzip'})

In [ ]:
data_df = pd.read_csv(
    './data.csv', 
    sep = ';',
#    usecols = [col],
#    #parse_dates = ['StatTime'],
    header = None, 
    names = names, 
#    dtype = data_types_dict, 
#    nrows = 10_000
    skiprows = 10_000_000
    )

In [ ]:
data_train